In [ ]:
import PyPDF2
import re

import glob

from datetime import datetime

import pandas as pd

In [ ]:
# some regexes
datere = re.compile('[0-9]{4}\/[0-9]{2}\/[0-9]{2}\s+A\s+[0-9]{4}\/[0-9]{2}\/[0-9]{2}')
valre = re.compile('([1-9]{1,2}\.[0-9]{2}\s+[1-9]{1,2}\.[0-9]{2}\s+[0-9A-Z\s\:\.]{1,}[0-9]{1,2}\,[0-9]{2})')

In [ ]:
values = []
for pdf_file in glob.glob('meals/*'):
    print('processing ' + pdf_file)
    
    # open and decrypt file
    pdfFileObj = open(pdf_file, 'rb')
    pdfReader = PyPDF2.PdfFileReader(pdfFileObj)
    pdfReader.decrypt('')
    
    # some needed variables
    start_date = None
    end_date = None
    pages = []
    
    # iterate through the pages
    for page in range(pdfReader.numPages):
        pageObj = pdfReader.getPage(page)
        pages.append(pageObj.extractText())
        
    text = ' '.join(pages)
    
    # get the dates
    m = datere.search(text)
    fields = m.group().split()
    start_date = datetime.strptime(fields[0], '%Y/%m/%d')
    end_date = datetime.strptime(fields[2], '%Y/%m/%d')
            
    # get the values
    for val in valre.split(text):
        if val[0].isdigit():
            fields = [v for v in val.split('  ') if v]
            
            mdate = fields[0].split('.')
            vdate = fields[1].split('.')
            
            value = fields[4].replace(',', '.')
            if fields[3] == 'COMPRAS':
                value = 0 - float(value)
            else:
                value = float(value)
            
            
            values.append({
                    'movement_date': datetime(start_date.year, int(mdate[0]), int(mdate[1])),
                    'value_date': datetime(start_date.year, int(vdate[0]), int(vdate[1])),
                    'description': fields[2],
                    'ammount': value,
                    'balance': 0,
                    'available_balance': 0,
                    'movement_type': 'Débito' if fields[3] == 'COMPRAS' else 'Crédito',
                    'account_id': 30
                })
        

In [ ]:
df.sort_values(by='movement_date', inplace=True)
df.reset_index(inplace=True)

In [ ]:
balance = 0
bal_series = []
for index, row in df.iterrows():
    balance += row.ammount
    bal_series.append(balance)


#df.balance = df.shift(1).balance + df.ammount
df.balance = pd.Series(bal_series)
df.available_balance = df.balance

In [ ]:
df.drop('index', 1).to_csv('meals.csv', index=False)

In [ ]:
!cat meals.csv